In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
# from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re

data = pd.read_csv('eng_tweets_with_sent_RNN.csv')
data.head()
# Keeping only the neccessary columns
# data = data[['text','sentiment']]

data = data[['Company','Sentiment']]

data



,Company,Sentiment
0,Occipital,1
1,Occipital,-1
2,Occipital,1
3,Occipital,1
4,Occipital,0
...,...,...
177936,OvaScience,1
177937,OvaScience,1
177938,OvaScience,-1
177939,OvaScience,1


In [2]:
#In this Bidirectional LSTM model, we have replaced the LSTM layer from the previous model with a Bidirectional LSTM layer. The Bidirectional LSTM layer processes the input sequence in both forward and backward directions, which can capture dependencies in the sequence more effectively. We have also added a second dense layer after the Bidirectional LSTM layer to increase the model capacity, and a Dropout layer to reduce overfitting.

import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense, Dropout

# Load the data
#data = pd.read_csv('path/to/data.csv')

# Preprocess the text
data['Company'] = data['Company'].apply(lambda x: x.lower())
data['Company'] = data['Company'].apply((lambda x: re.sub('[^a-zA-z0-9\s]','',x)))

# Split the data
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

# Tokenize the data
tokenizer = Tokenizer(num_words=2000, split=' ')
tokenizer.fit_on_texts(train_data['Company'].values)

X_train = tokenizer.texts_to_sequences(train_data['Company'].values)
X_train = pad_sequences(X_train)

X_test = tokenizer.texts_to_sequences(test_data['Company'].values)
X_test = pad_sequences(X_test, maxlen=X_train.shape[1])

# Categorize the sentiment labels
y_train = train_data['Sentiment'].apply(lambda x: 0 if x < 0 else (1 if x == 0 else 2))
y_train = to_categorical(y_train)

y_test = test_data['Sentiment'].apply(lambda x: 0 if x < 0 else (1 if x == 0 else 2))
y_test = to_categorical(y_test)

# Define the model
model = Sequential()
model.add(Embedding(2000, 128, input_length=X_train.shape[1]))
model.add(Bidirectional(LSTM(196, dropout=0.2, recurrent_dropout=0.2)))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100, batch_size=32)


Epoch 1/100
4449/4449 [==============================] - 384s 80ms/step - loss: 1.0302 - accuracy: 0.4968 - val_loss: 1.0181 - val_accuracy: 0.5037
Epoch 2/100
4449/4449 [==============================] - 238s 54ms/step - loss: 1.0162 - accuracy: 0.5026 - val_loss: 1.0159 - val_accuracy: 0.5051
Epoch 3/100
4449/4449 [==============================] - 234s 53ms/step - loss: 1.0127 - accuracy: 0.5036 - val_loss: 1.0165 - val_accuracy: 0.5016
Epoch 4/100
4449/4449 [==============================] - 238s 53ms/step - loss: 1.0102 - accuracy: 0.5051 - val_loss: 1.0160 - val_accuracy: 0.5029
Epoch 5/100
4449/4449 [==============================] - 233s 52ms/step - loss: 1.0088 - accuracy: 0.5053 - val_loss: 1.0158 - val_accuracy: 0.5032
Epoch 6/100
4449/4449 [==============================] - 196s 44ms/step - loss: 1.0076 - accuracy: 0.5067 - val_loss: 1.0186 - val_accuracy: 0.5029
Epoch 7/100
4449/4449 [==============================] - 191s 43ms/step - loss: 1.0064 - accuracy: 0.5069 - val_

4449/4449 [==============================] - 103s 23ms/step - loss: 1.0020 - accuracy: 0.5083 - val_loss: 1.0253 - val_accuracy: 0.5019
Epoch 57/100
4449/4449 [==============================] - 112s 25ms/step - loss: 1.0015 - accuracy: 0.5085 - val_loss: 1.0269 - val_accuracy: 0.5009
Epoch 58/100
4449/4449 [==============================] - 110s 25ms/step - loss: 1.0023 - accuracy: 0.5076 - val_loss: 1.0266 - val_accuracy: 0.5013
Epoch 59/100
4449/4449 [==============================] - 108s 24ms/step - loss: 1.0016 - accuracy: 0.5082 - val_loss: 1.0281 - val_accuracy: 0.5011
Epoch 60/100
4449/4449 [==============================] - 113s 25ms/step - loss: 1.0018 - accuracy: 0.5086 - val_loss: 1.0269 - val_accuracy: 0.5009
Epoch 61/100
4449/4449 [==============================] - 109s 25ms/step - loss: 1.0017 - accuracy: 0.5083 - val_loss: 1.0269 - val_accuracy: 0.5015
Epoch 62/100
4449/4449 [==============================] - 123s 28ms/step - loss: 1.0015 - accuracy: 0.5091 - val_loss: 

In [3]:
# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print('Test accuracy:', accuracy)

Test accuracy: 0.5005760192871094


In [6]:
# Get the predicted sentiment for each company in the test set
y_pred = model.predict(X_test)

# Convert the predicted sentiment probabilities to sentiment labels
y_pred_labels = np.argmax(y_pred, axis=1)
sentiment_labels = ['-1', '0', '1']
y_pred_sentiments = [sentiment_labels[i] for i in y_pred_labels]

# Add the predicted sentiment labels to the test data
test_data['Sentiment_Pred'] = y_pred_sentiments

# Group the test data by company and calculate the overall sentiment for each company
company_sentiments = test_data.groupby('Company')['Sentiment_Pred'].value_counts(normalize=True).unstack().fillna(0)

# Print the overall sentiment for each company
print(company_sentiments)
# Save the company_sentiments DataFrame to a CSV file
company_sentiments.to_csv('Company_Overall_Sentiment.csv')



1113/1113 [==============================] - 7s 7ms/step
Sentiment_Pred   -1    0    1
Company                      
140proof        0.0  0.0  1.0
1stdibs         0.0  0.0  1.0
2c2p            0.0  0.0  1.0
2tor            0.0  0.0  1.0
365datacenters  0.0  0.0  1.0
...             ...  ...  ...
zya             0.0  0.0  1.0
zymesolutions   0.0  1.0  0.0
zymeworks       0.0  1.0  0.0
zyncro          0.0  0.0  1.0
zynstra         0.0  0.0  1.0

[1714 rows x 3 columns]


In [7]:
# read in the two dataframes
df1 = pd.read_csv('Company_Overall_Sentiment.csv')
df2 = pd.read_csv('Company_Series.csv')

# merge the dataframes based on the common column "company"
merged_df = pd.merge(df1, df2, on='Company')

# view the merged dataframe
print(merged_df.head())

    Company   -1    0    1 Series_Type
0  140proof  0.0  0.0  1.0    Series_A
1      2c2p  0.0  0.0  1.0    Series_C
2      36kr  0.0  0.0  1.0    Series_C
3     4home  0.0  0.0  1.0    Series_A
4   6fusion  0.0  0.0  1.0    Series_B


In [8]:
merged_df.to_csv('FINAL_RESULT.csv')